In [ ]:

import sys, os
sys.path.append('./pyFM/') # UNCOMMENT THIS LINE IF YOU HAVE NOT INSTALLED pyFM (pip install pyfmaps)
sys.path.append('./VisualizationTools/')  # This is for visualization purposes only, not necessary to run the algorithms

import numpy as np



import VisualizationTools as plu
import pyFM

from pyFM.mesh import TriMesh
import pyFM.spectral as spectral

import DiscreteOpt

In [ ]:
def generate_init_map(mesh1, mesh2, k):
    """
    Simple function which generates an initial map between two meshes using a small sized functional map.
    This uses the fast the two meshes are in one-to-one correspondence with the same vertex order.

    You should replace this with your own initial map generation method, typically using nearest neighbor or landmarks.
    """
    FM_12 = spectral.mesh_p2p_to_FM(np.arange(mesh2.n_vertices), mesh1, mesh2, dims=k)
    p2p_21_init = spectral.mesh_FM_to_p2p(FM_12, mesh1, mesh2, n_jobs=-1)
    return p2p_21_init

# 1 - LOAD DATA 

We load, center, area-normalize the data, and compute the eigendecomposition of the laplacian on each shape.

In [ ]:
mesh1 = TriMesh("data/tr_reg_000.off", area_normalize=True, center=True).process(intrinsic=True)
mesh2 = TriMesh("data/tr_reg_024.off", area_normalize=True, center=True).process(intrinsic=True)

# 2 - GENERATE INITIAL MAP 

In [ ]:
p2p_12_init = generate_init_map(mesh2, mesh1, 10)
p2p_21_init = generate_init_map(mesh1, mesh2, 10)

# 3 - Discrete Optimization

We can simply generate the model to compute maps between the two.

In [ ]:
model = DiscreteOpt.DiscreteOptimization(mesh1, mesh2)

## 3.1 Parameters

Since the number of parameters can be overwhelming, there are multiple ways to define parameters for the model:
  1. by using a pre-existing version of the parameters in [DiscreteOpt/utils/params](./DiscreteOpt/utils/params/). These are yaml files which contains all the informations.   
  2. by providing the path to a yaml file with the parameters
  3. by providing dictionaries of parameters equivalent to a yaml file


There are also some functions to simply use an our pre-existing version of parameters and add changes.

The parameters are divided in two subset of parameters:
  - `sp_params` which define which spectral energies are to be minimized
  - `opt_params` which define optimization parameters such as number of iteration, upsampling steps, ...

Check teh yaml files for more info

In [ ]:
# Version 1
model.set_params("zoomout")  # Select "zoomout" or "bijective_zoomout"

# Version 2
model.set_params(params="./DiscreteOpt/utils/params/zoomout.yml")

# Version 3
# Lets generate sp_params and opt_params from existing things
from DiscreteOpt.utils.params_utils import generate_sp_params_from_template, get_default_opt_params

sp_params = generate_sp_params_from_template('bijective_zoomout', conf_weight=1e-1)  # Using parameters from bijective zoomout, but adding conformal energy with weight 1e-1
opt_params = get_default_opt_params()  # Let‘s get default optimization parameters
model.set_params(sp_params=sp_params, opt_params=opt_params)

# Let's see the parameters
print(model.sp_params, model.opt_params)

## 3.2 - Fitting the model

In [ ]:
model.set_params("bijective_zoomout")  # Select "zoomout" or "bijective_zoomout"

model.solve_from_p2p(p2p_21=p2p_21_init, p2p_12=p2p_12_init, n_jobs=20, verbose=True)

## 3.3 - Obtaining the pointwise map

The model works but seems to inverse front and back in the middle of the shape.

In [ ]:
p2p_21 = model.p2p_21

In [ ]:
plu.plot_p2p(mesh1, mesh2, p2p_21, pretty=True)

# 4 - Smooth Discrete Optimization

In [ ]:
model = DiscreteOpt.SmoothDiscreteOptimization(mesh1, mesh2)

## 4.1 Parameters

Since the number of parameters can be overwhelming, there are multiple ways to define parameters for the model:
  1. by using a pre-existing version of the parameters in [DiscreteOpt/utils/params](./DiscreteOpt/utils/params/). These are yaml files which contains all the informations.   
  2. by providing the path to a yaml file with the parameters
  3. by providing dictionaries of parameters equivalent to a yaml file


There are also some functions to simply use an our pre-existing version of parameters and add changes.

The parameters are divided in **three** subsets of parameters:
  - `sp_params` which define which spectral energies are to be minimized
  - `sm_params` which define which smoothness energies are to be minimized
  - `opt_params` which define optimization parameters such as number of iteration, upsampling steps, ...

Check teh yaml files for more info

In [ ]:
model.set_params("zoomout_rhm")  # Select "zoomout_rhm" or "zoomout_dirichlet" usually. You can try "zoomout_arap" and "zoomout_nicp" also
model.solve_from_p2p(p2p_21=p2p_21_init, p2p_12=p2p_12_init, n_jobs=20, verbose=True)

## 3.3 - Obtaining the pointwise map

Notice that the inversion disappeared

In [ ]:
p2p_21 = model.p2p_21

In [ ]:
plu.plot_p2p(mesh1, mesh2, p2p_21, pretty=True)